# Heat - Basics and Advanced Features

In [1]:
from IPython.display import HTML, IFrame, Image

## Heat Introduction

### Heat is an Orchestration service of OpenStack

* Prepare infrastructure for your cloud application
  * Some support of actual applicaiton deployment
* Automate OpenStack API calls
* Human- and machine-readable representation
* Treat your infrastructure as code

**Whatever you can make with a series of OpenStack API calls - it can be done with Heat!**

* API calls for CRUD operations on OpenStack services entities
* calls are made in correct order when needed

## Heat architecture

In [2]:
heat_arch={'url': "https://docs.google.com/drawings/d/1e_BOmSe9L89azynhZn3G93it0sxUVqewKKyxewforDg/pub?w=640&h=480"}

In [3]:
Image(**heat_arch)

### API Services

+ **heat-api** - REST API endpoint for OpenStack API
+ **heat-api-cfn** - REST API compatible with AWS CFN API
+ **heat-api-cloudwatch** - specialized API where ``heat-cfntools`` posts in-instance metrics


### heat-engine - does most of the work

+ parses template and builds a graph of dependencies
+ generates internal resources representation
+ executes the graph calling OpenStack clients' APIs
+ horizontally scalable
  - multiple workers per engine process
  - multiple engines (on different hosts)

## Template formats

### 3 supported formats

* AWS-compatible
* AWS yaml representation
* Native Heat Orchestration Template (HOT)

### AWSTemplateFormatVersion

* Historically the first supported format
* Written in JSON
* Format is compatible with AWS CloudFormation
  * Your (simple) templates will run both on AWS CFN and OpenStack

```json
{
  "AWSTemplateFormatVersion": "2010-09-09",
  "Description": "Example AWS CFN template",
  "Parameters" : {
    "KeyName" : {
      "Type" : "String"
    }
  },
  "Resources": {
    "CfnLBUser" : {
      "Type" : "AWS::IAM::User"
    },
    "CfnLBAccessKey" : {
      "Type" : "AWS::IAM::AccessKey"
    }
  }
}
```

### HeatTemplateFormatVersion

* YAML version of AWS CFN templates
* created due to popular demand

```python
yaml.dumps(json.loads(template))
```

```yaml
HeatTemplateFormatVersion: 2012-12-12
Description: >
  Example Heat template (AWS-style)
Parameters:
  KeyName:
    Type: String
Resources:
  CfnLBUser:
    Type: AWS::IAM::User
  CfnLBAccessKey:
    Type: AWS::IAM::AccessKey
```

### Native HOT format

* written in YAML
* most of development and innovation happens here
  * not necessary to keep AWS compatibility

```yaml
heat_template_format: 2010-09-09
description: Example HOT template
parameters:
  key_name:
    type: string
resources:
  password:
    type: OS::Heat::RandomString
```

## Template anatomy

### Six top-level sections
* Template version specifier

* Resources (optional, but why would you? :) )
* Parameters (optional)
* Outputs (optional)
* Description (optional)
* Parameter groups (optional, for UI tools)

Resources are made optional so that in principle one could create an empty stack that can be populated later with `stack-update`. This is useful for tests that excersise the stacks API.

```yaml
heat_template_format: 2015-04-30
```

```yaml
parameters:
  image:
    type: string
    default: TestVM
    description: A secret image to boot
    label: Image
    hidden: True
    constraints:
    - custom_constraint: glance.image
      description: Must be a valid Glance image
 ```

```yaml
resources:
  my_server:
    type: OS::Nova::Server
    properties:
      image: { get_param: image }
      flavor: m1.small
```

```yaml
outputs:
  server_ip:
    description: Private IP of the server
    value: { get_attr: [ my_server, networks, private, 0 ] }
```

### Intrinsic functions

* `get_resource` - insert the UUID of the referenced resource
* `get_param` - insert the value of the specified parameter
* `get_attr` - insert an attribute of specified resource
  * can dig into complex attributes by key or index
* `get_file` - fetch the file from URI and insert verbatim
* `str_replace` - replaces substrings in a string with provided values
* `repeat` (*new in Kilo*) - repeat template snippets
* ... and some more


## Resources

* describe some entity in an OpenStack service
  * usually that entity has a UUID in some DB of some OS service
  * usually Heat's resource UUID is equal to the UUID of the thing it describes
* three types of resources supported
  * **AWS compatible ones** - `AWS::...::...`, API-compatible with AWS CFN
  * **Native OpenStack** - `OS::...:...`, no limits on API, most innovation here
  * **Custom resources** - can be written as `stevedore` plugins

### Resource description
* `type` - one of the available resource types
* `depends_on` - for explicit dependencies
* `metadata` - any key-value pairs
* `properties` - arguments to the API call, schema is resource specific
* `update_policy` - how to update resource, resource-specific
* `delete_policy` - `[ Delete | Retain | Snapshot ]`, resource-specific 

### Built-in service help

* `heat resource-type-list`
  * shows the list of all currently registered resource types
  * *caveat* - not all might be actually available to the user
* `heat resource-type-show <type::of::resource>`
  * shows a required structure of a resource as JSON template
  * includes description and limitations of resource's properties

### Resource attributes
* many resources have them
* allow referencing some property of the resource instead of its UUID
* accessed by `get_attr` function
* used as inputs for properties of other resources or stack outputs 
* can have complex map/list structure and be traversed according to it

#### Complex attribute example
`OS::Nova::Server` has `networks` attribute

* its actual schema is `{“public”: [ip1, ip2...], “private”: [ip3, ip4]}`
* a single IP can be accessed with
```yaml
value: { get_attr: [ server, networks, private, 0 ] }
```

## Environments

* Allow to customize a given template without changing it
* Can be global (set by the cloud operator)
* or local (provided by user)

```bash
heat stack-create my_stack -f my_template.yaml \
     -e my_env.yaml
```

### Provide template parameters
```yaml
parameters:
  KeyName: my_key
  Image: my_image
```

### Override resource types
```yaml
resource_registry:
   "OS::Metering::Alarm": "OS::Ceilometer::Alarm"
```

### Define custom templates for resources
```yaml
resource_registry:
  "AWS::RDS::DBInstance": "file:///etc/heat/templates/AWS_RDS_DBInstance.yaml"
```

### Override only single resource
```yaml
resource_registry:
  resources:
    my_old_db:
      "AWS::RDS::DBInstance": "http://<some_server>/AWS_RDS_DBInstance.yaml"
```

### Debug the template! (*new in Kilo*)

```yaml
resource_registry:
  resources:
    buggy_*_server:
      hooks: [ pre-create, pre-update ]
```

* stack-create will be paused before creating this resource
* to continue use
```bash
heat resource-signal <stack-name> buggy_app_server \
     -D "{unset_hook: pre-create}"
```

## Moar template tricks

### Nested stacks

* also called `template resources`
* parameters of nested template are properties of the resource
* outputs of nested template are attributes of the resource

#### my_nova.yaml
```yaml
heat_template_version: 2014-10-16

parameters:
  key_name:
    type: string
    description: Name of a KeyPair

resources:
  server:
    type: OS::Nova::Server
    properties:
      key_name: {get_param: key_name}
      flavor: m1.small
      image: ubuntu-trusty-x86_64
```

### Use file name as resource type

```yaml
resources:
  my_server:
    type: my_nova.yaml
    properties:
      key_name: my_key
```

### Define new resource types
```yaml
resources:
  my_server:
    type: My::Custom::Server
    properties:
      key_name: my_key
```
##### environment
```yaml
resource_registry:
  "My::Custom::Server": my_nova.yaml
```

### Get attributes from nested stacks
```yaml
resources:
  my_server:
    type: my_nova.yaml

outputs:
  test_out:
    value: {get_attr: my_server, resource.server, first_address}
```

### Set ID of the nested resource
```yaml
outputs:
  OS::Stack_id:
    value: { get_resource: server }
```

## DRY - Grouping resources

##### you need to create a bunch of similar resources
  * e.g. servers

## DO NOT COPY-PASTE !

*Use* **OS::Heat::ResourceGroup**

```yaml
resources:
  my_group:
    type: OS::Heat::ResourceGroup
    count: 3
    type: OS::Nova::Server
    properties:
      name: my_server_%index%
      image: Fedora
      flavor: m1.small
```

##### and what if you need to create and manage bundles of resources?
  * e.g. server + volume + volume attachment

## DO NOT COPY-PASTE !

*Use* **OS::Heat::ResourceGroup** *AND* **environments**

##### parent.yaml
```yaml
resources:
  group:
    type: OS::Heat::ResourceGroup
    count: { get_param: count }
    resource_def:
      type: My::Awesome::Server
      properties:
        volsize: 1
        image: TestVM
        flavor: m1.small
```

##### nested.yaml
```yaml
parameters:
  image:
    type: string
  volsize:
    type: number
  flavor:
    type: string
resources:
  server:
    type: OS::Nova::Server
  ...
  volume:
    type: OS::Cinder::Volume
  ...
  server_volume:
    type: OS::Cinder::VolumeAttachment
  ...
```

##### registry.yaml
```yaml
resource_registry:
  "My::Awesome::Server": nested.yaml
```

```bash
heat stack-create nested -f parent.yaml \
     -e registry.yaml
```

#### somewhat more elaborate example:

* Neutron Network + Subnet + Router + Security Group
* Config of the webapp server
* A group of:
  * server with webapp
  * volume
  * volume attachment to the server
  * floating IP for the server
  
https://github.com/pshchelo/stackdev/tree/master/templates/sanity

## Deploying software with Heat

### Problem
* Nova declares the VM to be `ACTIVE` as soon as OS starts to boot

But

* your template pieces might depend on software actually running on VM

or

* you watch the heat stack and want to use the app as soon as stack is `CREATE_COMPLETE`

### Old good way - WaitCondition

* creates a presigned URL
  * on Heat API
  * or as Swift TempUrl
* make a `PUT` to this URL to register a success or failure
  * also can include any relevant data

```yaml
resources:
  handle:
    type: OS::Heat::WaitConditionHandle
  waiter:
    type: OS::Heat::WaitCondition
    properties:
      handle: { get_resource: handle }
      count: 1
      timeout: 600
```

##### ..continued
```yaml
  db_server:
    type: OS::Nova::Server
    properties:
      ...
      user_data_format: RAW
      user_data:
        str_replace:
          template: |
            #!/bin/bash -v
            <DO YOUR STUFF HERE>
            wc_notify -d '{"status": "SUCCESS", "reason": "<SOMEREASON>", "data": "SOMEDATA"}'
          params:
            wc_notify: { get_attr: [ handle, curl_cli ] }
  app_server:
    type: OS::Nova::Server
    depends_on: waiter
    ...
```

You can later retrieve the data passed to WaitCondition as `data` atribute of that resource.

All the fields in request are optional, `status` defaults to `SUCCESS`

### New shiny way - SoftwareConfig / SoftwareDeployment

## Where to go from here?

* https://github.com/openstack/heat-templates
* https://github.com/openstack/heat/tree/master/heat_integrationtests
* http://docs.openstack.org
  * http://docs.openstack.org/developer/heat
* openstack / openstack-dev ML - [Heat] tag
* \#heat @freenode.net
  * `pas-ha` *or* `pshchelo` is myself :)
* \#heat @miracloud.slack.com
* https://github.com/pshchelo/stackdev